In [38]:
import fastfusion
from fastfusion.mapper import Metrics

spec = fastfusion.Specification.from_yaml(
    "architectures/fastfusion/tpu_like.yaml",
    "workloads/fastfusion/gemm_16k.yaml",
)
spec.mapper.ffm.metrics = Metrics.ENERGY | Metrics.LATENCY
pmappings = fastfusion.mapper.FFM.make_pmappings(spec)
mappings = fastfusion.mapper.FFM.join_pmappings(spec, pmappings)

WARNING Loading configuration file from /home/gilbertm/work/venv/fastfusion/config.yaml
Generating pmapping templates for compute scalar_unit Einsum Matmul: 0it [00:00, ?it/s]           | 0/2 [00:00<?, ?it/s]
Generating pmapping templates for compute MAC Einsum Matmul: 8it [00:00, 175.69it/s]
Generating jobs: 100%|████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 29.14it/s]
WARNING Einsum Matmul has 8 pmapping templates:
WARNING 	[WA,I,A in MainMemory] T-m,na initial=symbol stride=symbol [A in GlobalBuffer] T-ni initial=symbol stride=symbol [I in GlobalBuffer] T-na initial=symbol stride=symbol S-Z-m initial=symbol stride=symbol S-Y-ni initial=symbol stride=symbol S-X-na initial=symbol stride=symbol [WA in Register] T-m initial=symbol stride=symbol MAC computes Matmul
WARNING 	[WA,I,A in MainMemory] T-m,ni initial=symbol stride=symbol [I in GlobalBuffer] T-na initial=symbol stride=symbol [A in GlobalBuffer] T-ni initial=symbol stride=symbol S-Z-m in

Einsum Matmul has 15 pmappings with 8 compatibilities


Compressing pmappings: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 221.45it/s]


Not tracking GlobalBuffer because it is never reserved for multiple pmappings.
Not tracking LocalBuffer because it is never reserved for multiple pmappings.
Not tracking MainMemory because it is never reserved for multiple pmappings.
Not tracking Register because it is never reserved for multiple pmappings.


Grouping pmappings: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 553.12it/s]


In [64]:
chosen_mapping = mappings.data.iloc[0]['Total<SEP>mapping']

In [65]:
mappings.data.iloc[0]

Total<SEP>latency                                                                        0.139461
Total<SEP>energy                                                                         0.586341
Matmul<SEP>stride0                                                                            128
Matmul<SEP>stride1                                                                           4096
Matmul<SEP>stride2                                                                            128
Matmul<SEP>stride3                                                                            128
Matmul<SEP>stride4                                                                             32
Matmul<SEP>latency<SEP>compute                                                           0.067109
Matmul<SEP>latency<SEP>Register                                                                 0
Matmul<SEP>latency<SEP>GlobalBuffer                                                      0.138412
Matmul<SEP>latency<S

In [66]:
from IPython.display import SVG, display

chosen_mapping().nodes[0].nodes[2]

Temporal(type=None, rank_variable='m', tile_pattern=TilePattern(type=None, stride=512, initial_tile_shape=None, calculated_n_iterations=32), assume_perfect_factor=True)